<a href="https://colab.research.google.com/github/Einsuomi/Python/blob/main/Forecast%20Hotel%20Ratings%20for%20Future%20reservations/ANN%20radisson%20blu%20sea%20side.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Radisson Blu Seaside Hotel, Helsinki.csv to Radisson Blu Seaside Hotel, Helsinki.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools


In [ ]:
import io
data = pd.read_csv(io.BytesIO(uploaded['Radisson Blu Seaside Hotel, Helsinki.csv']))

In [ ]:
data.head()

,Review_No,Page,Rating,Name,Country,Reviewed_Date,Room_Type,Nights,Check_In_Month,Travel_Type,Review_Details
0,1,1,10.0,Muhamad,Malaysia,Reviewed: 3 December 2021,Superior Room,1,November 2021,Group,A very pleasant stay Liked · Nice place to...
1,2,1,9.0,Nadiya,Czech Republic,Reviewed: 27 February 2023,Standard Room,1,February 2023,Couple,Superb Liked · We had a lovely room with h...
2,3,1,9.0,Alexandre,Brazil,Reviewed: 26 February 2023,Standard Room,3,February 2023,Family,"Superb Liked · Staff is very nice, room wa..."
3,4,1,8.0,Helen,United Kingdom,Reviewed: 21 February 2023,Standard Room,3,February 2023,Group,"Fantastic Liked · Fantastic location, shor..."
4,5,1,9.0,Asia,United Kingdom,Reviewed: 17 February 2023,Standard Room,1,February 2023,Solo traveller,Nice clean hotel with friendly people Liked...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier



from sklearn.model_selection import GridSearchCV


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score




# Importing required libraries

from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, recall_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier








# 1. Data Cleaning and Data Preprocessing

In [ ]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5464 entries, 0 to 5692
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Review_No       5464 non-null   int64  
 1   Page            5464 non-null   int64  
 2   Rating          5464 non-null   float64
 3   Name            5464 non-null   object 
 4   Country         5464 non-null   object 
 5   Reviewed_Date   5464 non-null   object 
 6   Room_Type       5464 non-null   object 
 7   Nights          5464 non-null   int64  
 8   Check_In_Month  5464 non-null   object 
 9   Travel_Type     5464 non-null   object 
 10  Review_Details  5464 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 512.2+ KB


In [ ]:
data_mm = data.drop(["Review_No","Page","Name","Reviewed_Date","Review_Details"],axis=1)

### 1.1 Preprocessing the categorical value in Country

In [ ]:
# 2. based on value_counts()

filter = data_mm.Country.value_counts()
data_mm.loc[data_mm.Country.isin(filter[filter<300].index),"Country"] = "Others"

data_mm.Country.value_counts()

Finland    2761
Others     2294
Germany     409
Name: Country, dtype: int64

### 1.2 Preprocessing the categorical value in Check_In_Month

In [ ]:
data_mm["Check_In_Month"]=data_mm["Check_In_Month"].map(lambda x: x[:-5])
data_mm["Check_In_Month"]=data_mm["Check_In_Month"].str.replace("\xa0","",regex=True)
data_mm.head()

,Rating,Country,Room_Type,Nights,Check_In_Month,Travel_Type
0,10.0,Others,Superior Room,1,November,Group
1,9.0,Others,Standard Room,1,February,Couple
2,9.0,Others,Standard Room,3,February,Family
3,8.0,Others,Standard Room,3,February,Group
4,9.0,Others,Standard Room,1,February,Solo traveller


### 1.3 Preprocessing the categorical value in Nights

In [ ]:
# Nights

filter_nights = data_mm.Nights.value_counts()
data_mm.loc[data_mm.Nights.isin(filter_nights[filter_nights<1000].index),"Nights"] = "More than 2 nights"
data_mm.loc[data_mm.Nights.isin(filter_nights[filter_nights>3000].index),"Nights"] = "1 night"

data_mm.loc[data_mm.Nights.isin(filter_nights[filter_nights==1413].index),"Nights"] = "2 nights"



### 1.4 Preprocessing the value of target variable Rating, Convert to categorical

In [ ]:
bins = [0,8.4,10.0]
class_name=[0,1]
rating_cat=pd.cut(data_mm.Rating,bins,labels=class_name)
data_mm["rating_cat"]=rating_cat

In [ ]:
data_mm=data_mm.drop(["Rating"],axis=1)


In [ ]:
data_mm.head()

,Country,Room_Type,Nights,Check_In_Month,Travel_Type,rating_cat
0,Others,Superior Room,1 night,November,Group,1
1,Others,Standard Room,1 night,February,Couple,1
2,Others,Standard Room,More than 2 nights,February,Family,1
3,Others,Standard Room,More than 2 nights,February,Group,0
4,Others,Standard Room,1 night,February,Solo traveller,1


# 2. Generate all possible feature subsets

In [ ]:
import itertools

# Get all possible subset of categorical columns to get_dummies.
cvar_list=["Country","Room_Type","Travel_Type","Check_In_Month","Nights"]
cvar_list_subsets = []
for i in range(0, len(cvar_list) + 1):
    cvar_list_subsets += [list(x) for x in itertools.combinations(cvar_list, i)]

# print the subsets
print(cvar_list_subsets)
print (len(cvar_list_subsets))

[[], ['Country'], ['Room_Type'], ['Travel_Type'], ['Check_In_Month'], ['Nights'], ['Country', 'Room_Type'], ['Country', 'Travel_Type'], ['Country', 'Check_In_Month'], ['Country', 'Nights'], ['Room_Type', 'Travel_Type'], ['Room_Type', 'Check_In_Month'], ['Room_Type', 'Nights'], ['Travel_Type', 'Check_In_Month'], ['Travel_Type', 'Nights'], ['Check_In_Month', 'Nights'], ['Country', 'Room_Type', 'Travel_Type'], ['Country', 'Room_Type', 'Check_In_Month'], ['Country', 'Room_Type', 'Nights'], ['Country', 'Travel_Type', 'Check_In_Month'], ['Country', 'Travel_Type', 'Nights'], ['Country', 'Check_In_Month', 'Nights'], ['Room_Type', 'Travel_Type', 'Check_In_Month'], ['Room_Type', 'Travel_Type', 'Nights'], ['Room_Type', 'Check_In_Month', 'Nights'], ['Travel_Type', 'Check_In_Month', 'Nights'], ['Country', 'Room_Type', 'Travel_Type', 'Check_In_Month'], ['Country', 'Room_Type', 'Travel_Type', 'Nights'], ['Country', 'Room_Type', 'Check_In_Month', 'Nights'], ['Country', 'Travel_Type', 'Check_In_Month',

# 3. Split data, Train Models, Evaluate Models

In [ ]:
# from scikeras.wrappers import TensorFlowKerasClassifier


In [ ]:
import tensorflow as tf

# One Model with one feature subset at one time


n=31

# Iterate over each combination of column names
for e in cvar_list_subsets[31:]:

    print ("The ",n,"model: ")
    print ("Predictors include categorical variables",e)
    data_m_c = data_mm.copy()

    for var in e:
        dummies_set = pd.get_dummies(data_m_c[var], drop_first=True, prefix = var)
        data_m_c = pd.concat([data_m_c, dummies_set], axis = 1).drop(var, axis = 1)
    #     print (data_m_c)
    #     break
    # break

    cvar_list_r = [x for x in cvar_list if x not in e]

    # print ("e (predictors):", e )
    # print ("cvar_list:",cvar_list)
    # print (cvar_list_r)
    # print (len(cvar_list_r))
    # break

    data_m_c.drop(cvar_list_r,axis=1,inplace = True)

    print ("predictors and rating_cat for double check:")
    print (data_m_c.columns)
    # break

    # split data for with all columns.

    col_list = list(data_m_c.columns)
    col_list.remove("rating_cat")
    X = data_m_c[col_list]
    y = data_m_c["rating_cat"]


#     label_encoder = LabelEncoder()
#     for col in X.columns:
#         X[col] = label_encoder.fit_transform(X[col])

#     print (X)
#     break

#     scaler = StandardScaler()
#     X = scaler.fit_transform(X)

    # Define the ANN model
    def create_model(num_hidden_layers=1, num_nodes=16):

        # set random state = 42
        np.random.seed(42)
        tf.random.set_seed(42)

        model = Sequential()
        model.add(Dense(num_nodes, input_dim=X.shape[1], activation='relu'))
        for i in range(num_hidden_layers):
            model.add(Dense(num_nodes, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    # Wrap the Keras model in a scikit-learn classifier
    model = KerasClassifier(build_fn=create_model)

    # Define the hyperparameter grid to search over
    param_grid = {
        'batch_size': [30, 50],
        'epochs': [100, 200],
        'num_hidden_layers': [2, 6],
        'num_nodes': [2,6]
    }

    # Define the scoring metric
    def TN_score(y_test, y_pred):
        cm = confusion_matrix(y_test, y_pred)
        # tn, fp, fn, tp = cm.ravel()
        recall_0 = cm[0][0] / (cm[0][0] + cm[0][1])
        tn_fn_ratio = cm[0][0] / cm[1][0]
        return recall_0 * tn_fn_ratio

    my_scorer = make_scorer(TN_score, greater_is_better=True)

    # Create the GridSearchCV object
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=my_scorer, refit='TN_score')

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

    # Fit the GridSearchCV object to the data
    grid_result = grid.fit(X_train, y_train)



    # Print the best recall score and the corresponding hyperparameters

    # Evaluate the best model on the testing set
    y_pred = grid_result.best_estimator_.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    print ("the",n,":",e)
    print(cm)
    print(classification_report(y_test,y_pred))


    print("Best score : {:.2f} using {}".format((cm[0][0]/(cm[0][0]+cm[0][1]))*(cm[0][0]/ cm[1][0]), grid_result.best_params_))

    print ("\n")
    print ("*"*60)


    break


The  31 model: 
Predictors include categorical variables ['Country', 'Room_Type', 'Travel_Type', 'Check_In_Month', 'Nights']
predictors and rating_cat for double check:
Index(['rating_cat', 'Country_Germany', 'Country_Others',
       'Room_Type_Executive Suite', 'Room_Type_One-Bedroom Suite',
       'Room_Type_Standard Room', 'Room_Type_Superior Room',
       'Travel_Type_Family', 'Travel_Type_Group', 'Travel_Type_Solo traveller',
       'Check_In_Month_August', 'Check_In_Month_December',
       'Check_In_Month_February', 'Check_In_Month_January',
       'Check_In_Month_July', 'Check_In_Month_June', 'Check_In_Month_March',
       'Check_In_Month_May', 'Check_In_Month_November',
       'Check_In_Month_October', 'Check_In_Month_September', 'Nights_2 nights',
       'Nights_More than 2 nights'],
      dtype='object')


<ipython-input-19-f19da7ae2c13>:67: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


Epoch 1/100
124/124 [==============================] - 3s 5ms/step - loss: 0.6936 - accuracy: 0.5179
Epoch 2/100
124/124 [==============================] - 1s 9ms/step - loss: 0.6894 - accuracy: 0.5569
Epoch 3/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6880 - accuracy: 0.5577
Epoch 4/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6873 - accuracy: 0.5577
Epoch 5/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6869 - accuracy: 0.5577
Epoch 6/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6867 - accuracy: 0.5577
Epoch 7/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6866 - accuracy: 0.5577
Epoch 8/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6866 - accuracy: 0.5577
Epoch 9/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6865 - accuracy: 0.5577
Epoch 10/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6865 - accura

<ipython-input-19-f19da7ae2c13>:82: RuntimeWarning: invalid value encountered in long_scalars
  tn_fn_ratio = cm[0][0] / cm[1][0]


124/124 [==============================] - 1s 2ms/step - loss: 0.6851 - accuracy: 0.5635
Epoch 2/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6829 - accuracy: 0.5627
Epoch 3/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6807 - accuracy: 0.5627
Epoch 4/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6790 - accuracy: 0.5619
Epoch 5/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6770 - accuracy: 0.5654
Epoch 6/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6755 - accuracy: 0.5748
Epoch 7/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6742 - accuracy: 0.5770
Epoch 8/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6732 - accuracy: 0.5780
Epoch 9/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6728 - accuracy: 0.5810
Epoch 10/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6720 - accuracy: 0.5802
E

...

75/75 [==============================] - 2s 2ms/step - loss: 0.6917 - accuracy: 0.5600
Epoch 2/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6895 - accuracy: 0.5622
Epoch 3/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6880 - accuracy: 0.5622
Epoch 4/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6871 - accuracy: 0.5622
Epoch 5/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6864 - accuracy: 0.5622
Epoch 6/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6861 - accuracy: 0.5622
Epoch 7/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6858 - accuracy: 0.5622
Epoch 8/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6857 - accuracy: 0.5622
Epoch 9/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6856 - accuracy: 0.5622
Epoch 10/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6855 - accuracy: 0.5622
Epoch 11/200
75/75 [=

<ipython-input-19-f19da7ae2c13>:82: RuntimeWarning: invalid value encountered in long_scalars
  tn_fn_ratio = cm[0][0] / cm[1][0]


Epoch 1/200
75/75 [==============================] - 2s 3ms/step - loss: 0.6905 - accuracy: 0.5478
Epoch 2/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6862 - accuracy: 0.5577
Epoch 3/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6841 - accuracy: 0.5577
Epoch 4/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6823 - accuracy: 0.5577
Epoch 5/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6793 - accuracy: 0.5655
Epoch 6/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6780 - accuracy: 0.5728
Epoch 7/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6772 - accuracy: 0.5742
Epoch 8/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6763 - accuracy: 0.5798
Epoch 9/200
75/75 [==============================] - 0s 3ms/step - loss: 0.6755 - accuracy: 0.5744
Epoch 10/200
75/75 [==============================] - 0s 2ms/step - loss: 0.6750 - accuracy: 0.5750
Epoch 11/

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan 0.39215121        nan        nan        nan 0.41087238
        nan 0.38414693        nan 0.34392712        nan 0.4050503
        nan 0.35457743        nan 0.39724513]
  warnings.warn(


155/155 [==============================] - 2s 3ms/step - loss: 0.6904 - accuracy: 0.5441
Epoch 2/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6857 - accuracy: 0.5614
Epoch 3/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6826 - accuracy: 0.5614
Epoch 4/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6800 - accuracy: 0.5614
Epoch 5/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6775 - accuracy: 0.5614
Epoch 6/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6761 - accuracy: 0.5614
Epoch 7/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6746 - accuracy: 0.5614
Epoch 8/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6738 - accuracy: 0.5614
Epoch 9/200
155/155 [==============================] - 0s 3ms/step - loss: 0.6731 - accuracy: 0.5614
Epoch 10/200
155/155 [==============================] - 0s 2ms/step - loss: 0.6726 - accuracy: 0.5620
E